# Detect unattended baggage

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uclasystem/VQPy/blob/main/examples/unattended_baggage/demo.ipynb)

## Introduction

Unattended baggage could simply be an oversight on the part of the owner; or it could be intentionally left behind, containing dangerous materials that pose a threat to public safety. Either way, detection is necessary.

In this example, we aim to identify unattended baggages and generate results for triggering an alarm in a surveillance video.
A baggage is regarded as unattended if it applies to the circumstances below:

- The distance to its owner is further than `DISTANCE_THRESHOLD`.
- No one attends the baggage for a period of `t` consecutive seconds.

Note that in this example, the baggage is not regarded as unattended if there is another person, like a staff, is within the distance threshold.

## Environment setup

Python3.8 is recommended to avoid compatibility issues when installing YOLOX.

Please download video from [kaggle](https://www.kaggle.com/datasets/szahid405/baggage?select=baggage.mp4) and place it in the same directory as this notebook, aside from running the cell below.

In [ ]:
# install YOLOX
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
# download YOLOX pretrained model
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_x.pth
!cd YOLOX && pip3 install .
# download VQPy, move vqpy/ to root directory for import
!git clone https://github.com/uclasystem/VQPy.git
!mv VQPy/vqpy ./
# install VQPy's dependencies
!pip3 install lap cython_bbox shapely
import vqpy
video_path = "./baggage.mp4"	# or change to wherever you put the video
save_folder = "./vqpy_outputs"
detector_model_dir = "./"

For reference, the working directory should look like:

```text
.
├── VQPy
├── YOLOX
├── baggage.mp4	# video to query on
├── vqpy	# make vqpy available for import
└── yolox_x.pth	# model checkpoint
```

## Unattended baggage query with VQPy

### Step 1: Define interested `VObj`s

#### Define `Person` `VObj`

Since we will only use the built-in properties of `VObj` in `Person`, `Person` can directly inherit `VObjBase`, with no extra property functions defined.

In [ ]:
class Person(vqpy.VObjBase):
    pass

The built-in properties we will use for `Person` `VObj` are:

- bounding box (`tlbr`), for computing distance between person and baggage
- id (`track_id`), for referencing `Person` from baggage `VObj`

They will be used while computing the `owner` property of `Baggage` `VObj`, which we will introduce next.

#### Define VObj type for baggage with property `owner`

Next define a `Baggage` `VObj`. To query baggages without owners, we add a property `owner` to `Baggage`.

In this example, the baggage's `owner` references `track_id` of `Person` VObj, who is:

- the owner from last frame, if person is still present and within `DISTANCE_THRESHOLD`
- else, person closest to the baggage and within `DISTANCE_THRESHOLD`
- else, None

And we should use the below decorators to decorate `owner`:

- `@stateful`, to make values of `owner` from past frames available

	`@stateful(length=2)` tells VQPy to store value of property in the previous frame being decorated to the VObj, allowing us to use `getv("owner", -2)` to access value of `owner` of previous frame. `length` specifies number of past values to be saved, and the default `length=0` stores all past values.

- `@cross_vobj_property`, to retrieve information from other VObjs (`Person`s) in the current frame

	The arguments for the decorator are:

	- `vobj_type=Person`, type of VObj to request properties from
	- `vobj_input_fields=("track_id", "tlbr")`, names of properties

		`track_id` is the tracking id of `Person`, used to identify change of person around the baggage; `tlbr` is the bounding box of `Person`, a list containing coordinates of top-left and lower-right corners

	All properties requested will be passed to the function in form of a list of tuples, each tuple containing properties of a VObj. i.e. `[person_1_properties, person_2_properties, ..., person_n_peroperties]`, where `person_i_properties` is a tuple of the input field values: `(id, tlbr)`.

With property `owner`, we define `Baggage`:

In [ ]:
# helper function to calculate distance between two bounding boxes, in pixels
def distance(vobj1_tlbr, vobj2_tlbr):
    from math import sqrt

    center1 = (vobj1_tlbr[:2] + vobj1_tlbr[2:]) / 2
    center2 = (vobj2_tlbr[:2] + vobj2_tlbr[2:]) / 2
    # difference between center coordinate
    diff = center2 - center1
    return sqrt(diff[0] ** 2 + diff[1] ** 2)


class Baggage(vqpy.VObjBase):
    @vqpy.stateful(length=2)
    @vqpy.cross_vobj_property(vobj_type=Person, vobj_input_fields=("track_id", "tlbr"))
    def owner(self, person_ids_tlbrs):
        baggage_tlbr = self.getv("tlbr")
        prev_owner = self.getv("owner", -2)
        owner_id = None
        DISTANCE_THRESHOLD = (
            baggage_tlbr[3] - baggage_tlbr[1]
        ) + 1  # set threshold to baggage's width
        min_dist = DISTANCE_THRESHOLD + 1  # distance to nearest person
        # iterating through all people in the frame
        for person_id, person_tlbr in person_ids_tlbrs:
            dist = distance(baggage_tlbr, person_tlbr)
            if person_id == prev_owner and dist <= DISTANCE_THRESHOLD:
                # return previous owner if still around
                return prev_owner
            if dist <= DISTANCE_THRESHOLD and dist < min_dist:
                # update owner if closer
                owner_id = person_id
                min_dist = dist
        # new owner is returned (will return None if owner not found)
        return owner_id

### Step 2: Query on baggage's owner

#### Filter `Baggage`s without owner (`filter_cons`)

We regard baggage without an owner for more than 10 seconds as unattended, using `vqpy.utils.continuing(condition, duration, name)` with condition `lambda x: x is None` to filter property `"owner"`. Duration is set to `10` (seconds), and we use the name `no_owner` to notate the filter.

`vqpy.utils.continuing` will generate a property named `{filter_name}_periods` (`no_owner_periods` here) in the VObj. The property will have time periods across the whole video during which `condition` is satisfied for more than the `duration` specified. `{filter_name}_periods` can be used in `select_cons` as an output.

In [ ]:
filter_cons = {
    "__class__": lambda x: x == Baggage,
    "owner": vqpy.utils.continuing(
        condition=lambda x: x is None, duration=10, name="no_owner"
    ),
}

#### Select `Baggage`'s properties for output (`select_cons`)

For each baggage without owner, the following properties are selected for output:

- `track_id`, tracking id, built-in property of VObj
- `tlbr`, bounding box, also built-in property of VObj. Use post-processing function to convert to string before serializing.
- `no_owner_periods`, a list of time periods (in seconds) of `Baggage` having no owner

In [ ]:
select_cons = {
    "track_id": None,
    "tlbr": lambda x: str(x),
    "no_owner_periods": None,
}

#### The query

With `filter_cons` and `select_cons` composed, the query is:

In [ ]:
class FindUnattendedBaggage(vqpy.QueryBase):
    @staticmethod
    def setting() -> vqpy.VObjConstraint:
        return vqpy.VObjConstraint(
            filter_cons=filter_cons,
            select_cons=select_cons,
            filename="unattended_baggage",
        )

### Step 3: Running the query

With the `Person` VObj and the query defined, we can run the query, where:

- `cls_name` is a tuple for mapping numerical outputs of object detector to literal detection class name

	Here we use `COCO_CLASSES` since it includes all the class names of interest in the unattended baggage query, i.e. `"person", "backpack", "suitcase"`.

- dictionary `cls_type` is then used to map detection class name (in str) to VObj types defined

	`{"person": Person, "backpack": Baggage, "suitcase": Baggage}` means we wish to map COCO class `person` to VObj type `Person`, and `backpack, suitcase`  to `Baggage`

- `tasks` is a list of queries to run on the video

In [ ]:
vqpy.launch(
    cls_name=vqpy.COCO_CLASSES,
    cls_type={"person": Person, "backpack": Baggage, "suitcase": Baggage},
    tasks=[FindUnattendedBaggage()],
    video_path=video_path,
    save_folder=save_folder,
    detector_model_dir=detector_model_dir,
)

## Expected result

Result of the query will be in `{save_folder}/{video_name}_{task_name}_{detector_name}.json`, output for this example should be in `./vqpy_outputs/baggage_unattended_baggage_yolox.json`.

One entry is created for each frame that has filter condition satisfied.

At 3102th frame (around 103s), the last frame in the video that has unattended baggage(s), we have output:

```json
{
  "frame_id": 3102,
  "data": [
    {
      "track_id": 522,
      "tlbr": "[ 65.91797 244.125   147.65625 397.6875 ]",
      "no_owner_periods": [
        [49, 83],
        [86, 103]
      ]
    }
  ]
}
```

Time period `[86, 103]` in `no_owner_periods` means that the baggage is unattended during approximately 01:26-01:43.

Frame 2550, 2580 and 3102 looks like:

<img src="./demo.assets/frame2550.png" alt="frame2550" style="zoom: 40%;" />
<img src="./demo.assets/frame2580.png" alt="frame2580" style="zoom: 40%;" />
<img src="./demo.assets/frame3102.png" alt="frame3102, with bounding box" style="zoom: 40%;" />

As shown in frame 2550, the person in the blue box was regarded as the owner of the baggage prior to 01:26. The person left the frame in frame 2580, and the baggage did not have any owner during 86-103s, which is the interval `[86, 103]` in the output. The interval ends at frame 3102, when the other person marked with blue box approached the baggage and is regarded as its owner.